In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import os
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import base64
import os
from PIL import Image
from io import BytesIO
import re
import time
import requests
from bs4 import BeautifulSoup

def save_image_from_data_url(data_url, file_path):
    """data URL形式の画像をファイルに保存する"""
    header, encoded = data_url.split(',', 1)
    data = base64.b64decode(encoded)
    image = Image.open(BytesIO(data))
    image.save(file_path)

def get_img(driver, url, num, key):
    driver.get(url)

    # 緩やかにスクロールするJavaScript関数を定義
    scroll_js = """
    function slowScroll() {
        var scrollHeight = document.body.scrollHeight;
        var currentScroll = 0;
        var scrollIncrement = 10;  // 小さな増分でスクロールする（ピクセル）
        
        function scroll() {
            if (currentScroll < scrollHeight) {
                window.scrollBy(0, scrollIncrement);
                currentScroll += scrollIncrement;
                console.log(currentScroll);  // 現在のスクロール位置をコンソールに出力
                setTimeout(scroll, 50);  // 50ミリ秒ごとにスクロールする
            }
        }
        
        scroll();  // スクロール関数を実行
    }
    slowScroll();
    """

    # スクロールを実行
    driver.execute_script(scroll_js)
    time.sleep(5)  # ページが読み込まれるまで待つ

    html = driver.page_source.encode("utf-8")
    soup = BeautifulSoup(html, "html.parser")

    # 画像URLを格納するリスト
    target_list = []
    div_tags = soup.find_all('div', class_='leading-[0px] xl:hover:opacity-70')
    
    if not div_tags:
        print(f"No div tags found for class 'leading-[0px] xl:hover:opacity-70' on URL {url}")

    for div in div_tags:
        img = div.find('img')
        if img and 'src' in img.attrs:
            target_src = img['src']
            target_list.append(target_src)

    # 画像URLを保存するディレクトリ
    output_dir = f'C:/Users/NDF05/OneDrive/ドキュメント/VsCodeProject/Pytyon/torend/Select_imegMnager/wear_img2/{key}'
    os.makedirs(output_dir, exist_ok=True)

    # 画像をダウンロード
    if not target_list:
        print(f"No images found on URL {url}")
    
    for i, src in enumerate(target_list):
        file_path = os.path.join(output_dir, f'image_{num}_{i}.jpg')
        if src:
            try:
                if src.startswith('data:image'):
                    # data URL形式の画像処理
                    save_image_from_data_url(src, file_path)
                else:
                    # 通常のURLから画像をダウンロード
                    response = requests.get(src)
                    if response.status_code == 200:
                        with open(file_path, 'wb') as file:
                            file.write(response.content)
                    else:
                        print(f"Failed to download image from {src}")
            except Exception as e:
                print(f"Error saving image from {src}: {e}")

def start_page(key='地雷'):
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument('--window-size=1920x1080')
    options.add_argument('--no-sandbox')

    service = Service('/path/to/chromedriver')  # chromedriverのパスを指定
    driver = webdriver.Chrome()

    driver.get("https://wear.jp/")

    # 検索ボックスに「量産型」と入力して検索
    search_box = driver.find_element(By.CSS_SELECTOR, ".icon_font > input")
    search_box.click()
    search_box.send_keys(key)
    search_box.send_keys(Keys.ENTER)

    # 明示的な待機を追加して要素が表示されるまで待機
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'a')))

    # ページ全体のテキストを取得
    html = driver.page_source.encode("utf-8")
    soup = BeautifulSoup(html, "html.parser")

    # すべてのテキストを取得して確認
    all_text = soup.get_text()
    print(all_text)  # すべてのテキストを出力して確認

    # 正規表現パターンで数値を抽出
    matches = re.findall(r'\d+', all_text)

    # 最後の数値を取得
    if matches:
        number = int(matches[-1])  # 数値を整数型に変換
        print(f"見つかった数値: {number}")
    else:
        number = None
        print("数値が見つかりませんでした")

    current_url = driver.current_url
    print(current_url)

    converted_url = current_url.replace("coordinate/?tag_ids=", "women-coordinate/?pageno={}&tag_ids=")

    # 結果を表示
    print(converted_url)
    
    # number が None でない場合のみリストを作成
    list_URL = []
    if number:
        for i in range(1, number + 1):
            dynamic_url = converted_url.format(str(i))
            list_URL.append(dynamic_url)
    else:
        print("動的URLリストを生成できませんでした")

    # ドライバーを閉じる
    driver.quit()

    return number, list_URL, key

casual_styles = [
    "ベーシック",
    "アメリカンカジュアル",
    "モードカジュアル",
    "スポーツカジュアル",
    "ストリートカジュアル",
    "アウトドア"
]

elegant_styles = [
    "エレガンス",
    "ガーリー",
    "オフィスカジュアル",
    "キャリア",
    "マニッシュ",
    "デザイナーズ"
]

other_styles = [
    "トラッド",
    "ノームコア",
    "ナチュラル",
    "韓国",
    "量産型",
    "地雷系"
]

sum_list = casual_styles + elegant_styles + other_styles

for j in range(len(sum_list)):
    number, list_URL, key = start_page(sum_list[j])
    if number:
        time.sleep(2)
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--disable-gpu')
        options.add_argument('--window-size=1920x1080')
        options.add_argument('--no-sandbox')

        service = Service('/path/to/chromedriver')  # chromedriverのパスを指定
        driver = webdriver.Chrome()
        print(len(list_URL))
        for i in range(len(list_URL)):
            time.sleep(2)
            get_img(driver, list_URL[i], i, key)
        driver.quit()


「韓国」の人気ファッションコーディネート - WEARWEAR最新トレンドのコーデを見つけよう！アプリで開くログイン新規会員登録コーディネートやユーザーを探すコーディネートユーザーアイテム店舗候補韓国韓国韓国ファッション韓国ファッション韓国子供服韓国子供服韓国 冬韓国冬韓国ファッション 冬韓国ファッション冬韓国人韓国人韓国 ファッション韓国ファッション韓国風韓国風韓国旅行韓国旅行韓国 夏韓国夏韓国コーディネートユーザーアイテム店舗ログインまたは新規会員登録トップページランキングタイムラインマイページ基本設定メールマガジン設定プロフィール設定PROFILEプロフィールの変更はWEARアプリでできますアプリをダウンロードアプリを開く人気順絞り込み1人気順新着順すべて男性女性キッズコーディネートユーザーアイテム店舗コーディネート一覧（タグ：韓国）「韓国」コーディネート9,999+件絞り込みユーザー性別すべて性別男性女性キッズタイプすべてタイプWEARISTAショップスタッフ一般着用アイテムカテゴリー指定なしカテゴリートップスすべてのトップスTシャツ/カットソーシャツ/ブラウスビジネスシャツポロシャツニット/セーターベストパーカースウェットカーディガン/ボレロアンサンブルジャージタンクトップキャミソールチューブトップその他トップスジャケット/アウターすべてのジャケット/アウターテーラードジャケットノーカラージャケットノーカラーコートデニムジャケットライダースジャケットブルゾンミリタリージャケットMA-1ダウンジャケット/コートダウンベストダッフルコートモッズコートピーコートステンカラーコートトレンチコートチェスターコートムートンコートナイロンジャケットマウンテンパーカースタジャンスカジャンセットアップカバーオールポンチョその他アウターパンツすべてのパンツデニムパンツカーゴパンツチノパンツスウェットパンツスラックスその他パンツオールインワン・サロペットすべてのオールインワン・サロペットサロペット/オーバーオールつなぎ/オールインワンスカートすべてのスカートスカートデニムスカートワンピース/ドレスすべてのワンピース/ドレスワンピースシャツワンピースジャンパースカートチュニックドレスパンツドレスフォーマルスーツ/小物すべてのフォーマルスーツ/小物スーツジャケットスーツベストスーツ